# AI To Play Mine Sweeper

## The Game Object

I developed the pythonic minesweeper. It is unit tested fairly throughly; up until visualizations and tracking features of the project present themselves. Note: The 9's are bombs; because it is impossible for a cell to be sourounded by more Bombs then there are cells surronding it.

In [1]:
import game
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
mygame = game.minesweeper(14, 8, 25)


for row in mygame.board:
    for c in row:
        print(c, "\t", end="")
    print("\n")
#mygame.clickCell((0,0))

0.0 	0.0 	1.0 	9.0 	9.0 	2.0 	0.0 	1.0 	1.0 	2.0 	9.0 	1.0 	2.0 	9.0 	

0.0 	0.0 	2.0 	4.0 	9.0 	3.0 	1.0 	2.0 	9.0 	3.0 	1.0 	1.0 	2.0 	9.0 	

0.0 	0.0 	1.0 	9.0 	4.0 	9.0 	1.0 	2.0 	9.0 	2.0 	1.0 	1.0 	3.0 	2.0 	

0.0 	0.0 	2.0 	4.0 	9.0 	3.0 	1.0 	1.0 	1.0 	1.0 	1.0 	9.0 	3.0 	9.0 	

0.0 	1.0 	3.0 	9.0 	9.0 	2.0 	1.0 	1.0 	1.0 	1.0 	3.0 	3.0 	5.0 	9.0 	

0.0 	1.0 	9.0 	9.0 	3.0 	2.0 	2.0 	9.0 	2.0 	2.0 	9.0 	9.0 	3.0 	9.0 	

0.0 	2.0 	3.0 	3.0 	1.0 	1.0 	9.0 	3.0 	9.0 	2.0 	2.0 	2.0 	2.0 	1.0 	

0.0 	1.0 	9.0 	1.0 	0.0 	1.0 	1.0 	2.0 	1.0 	1.0 	0.0 	0.0 	0.0 	0.0 	



## Visualizing the Game

The following is a rough gui for the game. Feel free to mess around; when you loose all of the cells will disable and you will be alerted. When you win you will be alerted. Note: If you want to start a new game you will have to run the cell above because you need to make a new object to start over

In [3]:
mygame.playOnNoteBook()

## Neruel Network to Play Mine Sweeper (Classification Approach) 

We will attempt to play minesweeper by generating lots of games (which will be used as our data). 

Things that we'll need to do:

Pad the game board with some element.
This way we distiguish the edges of the game board (all of our data needs to have the same number of features) 

Represent "unknown squares" in our classification we will have some data that are "completed games" (the whole board is visible) and others where the game is 80% finished; 60%; 40% and 20%. (Our "Solve Percent" utility will help us with this)

In [4]:
mygame = game.minesweeper(14, 8, 25)
mygame.solvePercent(.2)
mygame.playOnNoteBook()

In [5]:
theData = []

for p in [.8, .6, .4, .2]:
    for i in range(1000):
        mygame = game.minesweeper(10, 10, 23)
        mygame.solvePercent(p)
        dataToAdd = np.pad(mygame.visible, (1), 'constant', constant_values=(-1))
        dataToAdd = np.where(dataToAdd=='*', 0, dataToAdd)
        theData.append(dataToAdd)
        
for i in range(1000):
    mygame = game.minesweeper(10, 10, 23)
    dataToAdd = np.pad(mygame.board, (1), 'constant', constant_values=(-1))
    theData.append(dataToAdd.astype('<U3'))


In [6]:
theRealData = []

for data in theData:
    innerData = data[1:-1, 1:-1]
    for index, x in np.ndenumerate(innerData):
        toAppend = []
        toAppend.append(x)
        toAppend.append(data[index[0], index[1]])
        toAppend.append(data[index[0], index[1]+1])
        toAppend.append(data[index[0], index[1]+2])
        toAppend.append(data[index[0]+1, index[1]])
        toAppend.append(data[index[0]+1, index[1]+2])
        toAppend.append(data[index[0]+2, index[1]])
        toAppend.append(data[index[0]+2, index[1]+1])
        toAppend.append(data[index[0]+2, index[1]+2])
        theRealData.append(toAppend)

(Comment this line out if you don't want to eliminate the unknown value prediction)

In [7]:
theRealData = np.array(theRealData)
theRealData = theRealData[theRealData[:, 0] != '?']

This next line will make it so that it's classifying not bomb (which is true) and bomb (false) -- so if a cell is a bomb it will give it a false value. Think of it as an "okay to click" classification.

In [8]:
theRealData[:, 0] = ['1.0' if r else '0.0' for r in (theRealData[:, 0] != '9.0')]
theRealData

array([['0.0', '-1', '-1', ..., '9.0', '?', '?'],
       ['0.0', '?', '?', ..., '?', '?', '?'],
       ['0.0', '-1', '?', ..., '-1', '?', '?'],
       ...,
       ['1.0', '9.0', '2.0', ..., '-1.', '-1.', '-1.'],
       ['0.0', '2.0', '3.0', ..., '-1.', '-1.', '-1.'],
       ['0.0', '3.0', '9.0', ..., '-1.', '-1.', '-1.']], dtype='<U3')

In [9]:
theRealData = np.where(theRealData=='?', 10, theRealData)
X, y = theRealData[:, 1:].astype(float).astype(str), theRealData[:, 0].astype(float).astype(str)

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [10]:
print(enc.categories_)
X = enc.transform(X).toarray()

[array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32')]


In [11]:
X.shape

(309403, 88)

In [12]:
y.shape

(309403,)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

The Hidden Layer "8" is inspired by the idea that each surrounding node has been one-hot-encoded into 12 features

In [14]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8), random_state=1)

In [15]:
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.9134697315455961


In [16]:
confusion_matrix(y_test, y_pred)

array([[11760,  5206],
       [ 3629, 81508]])

In [17]:
clf.predict_proba(X_test[0].reshape(1, -1))[0][1]

0.5206790862306504

# The AI that Plays Mine Sweeper

In [26]:
mygame = game.minesweeper(10, 10, 23)
c = 0
mygame.playOnNoteBook()


while (mygame.visible != "You have lost" and mygame.visible != "You have won!"):
    curbest = 0
    bestindex = (0, 0)
    data = np.array(mygame.visible).astype('<U3')
    data = np.pad(data, (1), 'constant', constant_values=(-1))
    data = np.where(data=='*', 0, data)
    innerData = data[1:-1, 1:-1]
    for index, x in np.ndenumerate(innerData):
        if x == "?":
            toAppend = []
            toAppend.append(data[index[0], index[1]])
            toAppend.append(data[index[0], index[1]+1])
            toAppend.append(data[index[0], index[1]+2])
            toAppend.append(data[index[0]+1, index[1]])
            toAppend.append(data[index[0]+1, index[1]+2])
            toAppend.append(data[index[0]+2, index[1]])
            toAppend.append(data[index[0]+2, index[1]+1])
            toAppend.append(data[index[0]+2, index[1]+2])
            toAppend = np.array(toAppend)
            toAppend = np.where(toAppend=='?', 10, toAppend)
            toAppend = toAppend.astype(float).astype(str)
            toAppend = toAppend.reshape(1,-1)
            toAppend = enc.transform(toAppend).toarray()
            if clf.predict_proba(toAppend)[0][1] > curbest:
                curbest = clf.predict_proba(toAppend)[0][1]
                bestindex = index
    print(curbest)
    mygame.gameboard[bestindex[0]][bestindex[1]].click()
    #mygame.clickCell(bestindex)
    c+=1
    print(c, bestindex)

print(mygame.visible)

0.7393431164415517
1 (0, 1)
0.7479015119291139
2 (1, 3)
0.7584815795754526
3 (0, 4)
0.9382672708146601
4 (1, 5)
0.9382672708146601
5 (3, 7)
0.9109084959551236
6 (2, 7)
0.9237828574559883
7 (1, 8)
0.8942741154925411
8 (3, 3)
0.8444609019832348
9 (3, 4)
0.773840552663671
10 (4, 7)
0.7354197284797102
11 (9, 0)
0.7666099493634249
12 (6, 2)
0.7563932265952411


'You have lost'

13 (8, 2)
You have lost


In [19]:
mygame.board

array([[0., 0., 0., 0., 1., 9., 9., 3., 9., 1.],
       [0., 0., 0., 0., 2., 4., 9., 3., 2., 2.],
       [0., 0., 0., 0., 2., 9., 3., 1., 1., 9.],
       [1., 1., 0., 1., 3., 9., 3., 1., 2., 2.],
       [9., 1., 0., 2., 9., 4., 9., 1., 1., 9.],
       [2., 3., 3., 4., 9., 3., 1., 2., 2., 2.],
       [1., 9., 9., 9., 2., 2., 1., 2., 9., 1.],
       [1., 3., 9., 3., 1., 2., 9., 3., 1., 1.],
       [2., 3., 3., 2., 2., 3., 9., 2., 0., 0.],
       [9., 9., 2., 9., 2., 9., 2., 1., 0., 0.]])

# Idea's to Improve:
### Mess with Hyper Params
### Mess with the solve function (make exact percentages etc)
### Mess with what we are classifying (We just want to know if something is a bomb or not...
### More Data surronding each point

# Messing with Hyper Params

In [20]:
#clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

#clf2.fit(X_train, y_train)

#y_pred = clf2.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [21]:
# This one is inspired by the one hot incoding (Each feature was split into 11 parts)
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(11), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [22]:
#import pickle

#filename = 'clf1.sav'
#pickle.dump(clf, open(filename, 'wb'))

#filename = 'clf2.sav'
#pickle.dump(clf2, open(filename, 'wb'))

#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

In [23]:
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8, 3), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [24]:
#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

# Messing with what we are classifying

In [25]:
np.unique(y.shape)

array([309403])